In [14]:
import pickle
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.models import load_model



In [15]:
def network():
    """
    Define the network
    :return:
    """
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(4))
    model.add(Activation('softmax'))

    return model

In [16]:
def train(file_path, model):

    x_,y_ = pickle.load( open(file_path, "rb" ) )
    random_state = 130
    X_train, x_validation, y_train, y_validation = train_test_split(x_, y_, train_size = 0.80,
                                                                    test_size = 0.2,
                                                                    random_state = random_state)
    # preprocess data
    X_normalized = np.array(X_train / 255.0 - 0.5 )
    label_binarizer = LabelBinarizer()
    y_one_hot = label_binarizer.fit_transform(y_train)

    model.summary()
    model.compile('adam', 'categorical_crossentropy', ['accuracy'])
    history = model.fit(X_normalized, y_one_hot, epochs=20, validation_split=0.2)

    model.save('traffic_light_classifier.h5')
    return history

In [17]:
def test(file_path, model):

    X_test,y_test = pickle.load( open(file_path, "rb" ) )

    # preprocess data
    X_normalized_test = np.array(X_test / 255.0 - 0.5 )
    label_binarizer = LabelBinarizer()
    y_one_hot_test = label_binarizer.fit_transform(y_test)

    print("Testing")

    metrics = model.evaluate(X_normalized_test, y_one_hot_test)
    for metric_i in range(len(model.metrics_names)):
        metric_name = model.metrics_names[metric_i]
        metric_value = metrics[metric_i]
        print('{}: {}'.format(metric_name, metric_value))

In [23]:
def test_an_image(file_path, model):
    """
    resize the input image to [32, 32, 3], then feed it into the NN for prediction
    :param file_path:
    :return:
    """

    desired_dim=(32,32)
    img = cv2.imread(file_path)
    img_resized = cv2.resize(img, desired_dim, interpolation=cv2.INTER_LINEAR)
    img_ = np.expand_dims(np.array(img_resized), axis=0)

#     predicted_state = model.predict_classes(img_)
    
    predict_x=model.predict(img_) 
    classes_x=np.argmax(predict_x,axis=1)

    return classes_x



In [24]:

if __name__ == "__main__":
    model = network()
    train_file = "./data/train.p"
    test_file = "./data/test.p"

    # Train the network
    train(train_file, model)

    # Test the network
    test(test_file, model=load_model('traffic_light_classifier.h5'))

    #---Test with a single image---#
    demo_flag = True
    file_path = './data/green.jpg'
    states = ['red', 'yellow', 'green', 'off']
    if demo_flag:
        predicted_state = test_an_image(file_path, model=load_model('traffic_light_classifier.h5'))
        for idx in predicted_state:
            print(states[idx])


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 10, 10, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 activation_12 (Activation)  (None, 5, 5, 32)          0         
                                                                 
 flatten_6 (Flatten)         (None, 800)               0         
                                                                 
 dense_6 (Dense)             (None, 4)                 3204      
                                                                 
 activation_13 (Activation)  (None, 4)                 0         
                                                      

In [27]:
import pandas as pd

In [28]:
dff = pd.DataFrame(states)
dff.to_csv('traffic_light_class_names.csv', header=False, index=False)